In [ ]:
import pandas as pd
from prisma_llm import AI
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.dates as mdates


In [ ]:
df = pd.read_csv("data/final/merged_cases_comments.csv")

In [ ]:
# show the number of records in the dataframe
print(f"Number of records in the dataframe: {len(df)}")

# show the first 5 records in the dataframe
print("First 5 records in the dataframe:")
print(df.head())

# show the columns in the dataframe
print("Columns in the dataframe:")
print(df.columns)

In [ ]:

ai = AI()

In [ ]:

def build_prompt(description):
    prompt = f"""
You are a UX expert specialized in classifying tech support cases.

Your task is to classify each case into **one of these two categories**:
- Usability
- Non-Usability

**Definitions**:
- **Usability**: Cases where the user struggles with understanding, finding, or operating features correctly. Examples include:
    - Difficulty locating documentation, manuals, or instructions
    - Difficulty finding settings, options, or buttons
    - Confusing or unclear user interface
    - Poor navigation, complicated workflows
    - Missing information that prevents the user from completing a task

- **Non-Usability**: Cases that are primarily about technical malfunctions, bugs, hardware failures, or network/system errors. Examples include:
    - Hardware defects (broken devices, malfunctioning sensors)
    - Software bugs or glitches (system crashes, error messages)
    - Firmware or software update failures
    - Network, server, or connectivity problems

**Important**:
- Focus on whether the case is about *ease of use* (Usability) or *technical malfunction* (Non-Usability).
- Based on the full case description, including any customer and support comments, make your best classification.
- **Always choose exactly one category**: either "Usability" or "Non-Usability."
- **Answer with only the category name. No explanations.**

---

Case: \"{description}\"

Answer:
"""
    return prompt

In [ ]:
def label_case(description):
    response = ai.chat(build_prompt(description))
    return response

In [ ]:
def label_cases_in_batches(df, batch_size=5000):
    total_cases = len(df)
    for start in range(0, total_cases, batch_size):
        end = min(start + batch_size, total_cases)
        batch = df.iloc[start:end].copy()

        # Apply the labeling function to the batch
        batch["class"] = batch["description"].apply(label_case)
        batch.to_csv(f"./llm/labeled_cases_{start}_{end}.csv", index=False)

        # Print progress and distribution
        print(f"✅ Labeled cases from {start} to {end} and saved to labeled_cases_{start}_{end}.csv")
        print(batch["class"].value_counts())
        print("----------")

In [ ]:
# Apply the function to the entire DataFrame
label_cases_in_batches(df, batch_size=5000)

In [ ]:
#combine all the labeled cases in the llm folder into one file only use 2 columns: case_number and class to create the new file
import pandas as pd
import glob

# Get all CSV files in the llm folder
csv_files = glob.glob("./llm/labeled_cases_*.csv")

# Initialize an empty list to store DataFrames
dataframes = []

# Loop through each file and read it into a DataFrame
for file in csv_files:
    df = pd.read_csv(file, usecols=["case_number", "class"])
    dataframes.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(dataframes, ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv("./llm/labeled_cases_combined.csv", index=False)

In [ ]:
# Now combine it again but with all columns
import pandas as pd
import glob


# Get all CSV files in the llm folder
csv_files = glob.glob("./llm/labeled_cases_*.csv")


# Initialize an empty list to store DataFrames
dataframes = []

# Loop through each file and read it into a DataFrame
for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(dataframes, ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv("./llm/labeled_cases_combined_all_columns.csv", index=False)



In [ ]:
# show me the count of each class in the combined file
combined_df = pd.read_csv("./llm/labeled_cases_combined_all_columns.csv")
combined_df["class"].unique()
# show all the unique values in the class column by increasing the pd.options.display.max_height
pd.options.display.max_rows = 1000
combined_df["class"].unique()


In [ ]:
# Show the count of each class in the combined file
combined_df["class"].value_counts()

In [ ]:
# Only keep the Usability and Not Usability classes
combined_df = combined_df[combined_df["class"].isin(["Usability", "Non-Usability"])]

# Show the count of each class in the combined file again
combined_df["class"].value_counts()

In [ ]:
# show number of records in combined_df
len(combined_df)


In [ ]:
# save the dataframe to a csv file named All cases labelled or something like that name
combined_df.to_csv("./llm/All_cases_labelled.csv", index=False)

# save it in an xlsx file too
combined_df.to_excel("./llm/All_cases_labelled.xlsx", index=False)

In [ ]:
# Save the Usability cases to a new CSV file and excel file
combined_df[combined_df["class"] == "Usability"].to_csv("./llm/usability_cases.csv", index=False)
combined_df[combined_df["class"] == "Usability"].to_excel("./llm/usability_cases.xlsx", index=False)

# Save the Non-Usability cases to a new CSV file and excel file
combined_df[combined_df["class"] == "Non-Usability"].to_csv("./llm/non_usability_cases.csv", index=False)
combined_df[combined_df["class"] == "Non-Usability"].to_excel("./llm/non_usability_cases.xlsx", index=False)

In [ ]:
# Set the style of seaborn
sns.set(style="whitegrid")

# Set the figure size
plt.figure(figsize=(10, 6))
# Set the font size
plt.rcParams.update({'font.size': 14})
# Set the font family
plt.rcParams.update({'font.family': 'Arial'})
# Set the font weight
plt.rcParams.update({'font.weight': 'bold'})
# Set the font size of the ticks
plt.tick_params(axis='both', which='major', labelsize=14)

# Set the font size of the legend
plt.rcParams.update({'legend.fontsize': 14})
# Set the font size of the title
plt.rcParams.update({'axes.titlesize': 16})
# Set the font size of the x and y labels
plt.rcParams.update({'axes.labelsize': 14})
# Set the font size of the x and y ticks
plt.rcParams.update({'xtick.labelsize': 14})

# Show the distribution of the classes in the combined file with percentages and count
combined_df["class"].value_counts(normalize=True).plot(kind="bar", color=["#1f77b4", "#ff7f0e"])
plt.title("Distribution of Usability and Non-Usability Cases")
plt.xlabel("Class")
plt.ylabel("Percentage")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("./llm/distribution_of_usability_and_non_usability_cases_percentage.png", dpi=300)
plt.show()


